In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy
import concurrent.futures

import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
import statsmodels.api as sm

import APIs.QuickStats as qs

import Utilities.SnD as us
import Utilities.Weather as uw
import Utilities.Modeling as um

import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [5]:
prec_units='in'
temp_units='F'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)

In [6]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)
model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [7]:
# Scenarios Creation
year_start = 2000
year_end = 2022

scenarios_EXT_dict = {}
for p in range(year_start,year_end):
    for t in range(year_start,year_end):
        scenarios_EXT_dict[str(p)+'_'+str(t)]={GV.WV_PREC:GV.EXT_ANALOG+'_'+str(p),GV.WV_SDD_30:GV.EXT_ANALOG+'_'+str(t)}

print(len(scenarios_EXT_dict))
# print(scenarios_EXT_dict)

484


In [8]:
%%time
results = {}
i=0
for sce_name, sce_dict in scenarios_EXT_dict.items():
    i+=1; print(i)
    pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units=prec_units, temp_units=temp_units, ext_mode = sce_dict)
    pred_df = cy.Build_Pred_DF(raw_data,milestones,pred_DF_instr,GV.CUR_YEAR, dt(2022,7,5))
    yields = model.predict(pred_df[model.params.index]).values

    results[sce_name]=yields[0]

[print(k,'->',v) for k,v in results.items()]
print(results)
print('All Done')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
results['2013_2012']

144.1662324890548

In [10]:
min(results.values())

144.1662324890548

In [11]:
min(results, key=results.get)

'2013_2012'